# CS 195: Natural Language Processing
## Handling Long-Term Information in Recurrent Neural Networks

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F6_4_LongTermRecurrence.ipynb)

## Reference

SLP: RNNs and LSTMs, Chapter 9 of Speech and Language Processing by Daniel Jurafsky & James H. Martin https://web.stanford.edu/~jurafsky/slp3/9.pdf

Wikipedia article on Gated Recurrent Unit: https://en.wikipedia.org/wiki/Gated_recurrent_unit

Sequence Classification with LSTM Recurrent Neural Networks in Python with Keras by Jason Brownlee: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [ ]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00


## Example Sentence

*The flights the airline was cancelling were full*

Suppose we have generated `The flights the airline`
* `was` is a good next choice
   - `airline` has context for `was` vs. `were`

Suppose we have generated `The flights the airline was cancelling`
* `was`/`were` depends on `flights`
* much more distance information

## The Vanishing Gradient

The *vanishing gradient* is a common problem in deep neural networks

If weights are < 1, they will get smaller and smaller each node they have to pass through - causing them to have little or not effect

Happens during training too - error/loss is propogated backwards through the network proportional to the weights on each edge
* earlier edges in the network are left with little error to use in adjusting weights

<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/vanishing_gradient.png?raw=1">
</div>

image source: https://www.researchgate.net/figure/A-visualization-of-the-vanishing-gradient-problem-using-the-architecture-depicted-in_fig8_277603865

## LSTM

Long Short-Term Memory (LSTM) networks try to address the vanishing gradient through
* removing unneeded information from the context
* adding information likely needed later

<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/LSTM_node.png?raw=1", width=700>
</div>

image_source: SLP Fig. 19.3, https://web.stanford.edu/~jurafsky/slp3/9.pdf

### How does it do this?

* Explicit **context layer** $c_t$
* neural **gates** that control the flow of information through the layer
    - $f$ - the **forget gate** - delete info from context that is no longer needed
    - $g$ - basic extraction of info from previous hidden state
    - $i$ - the **add gate** - select info to add to current context
    - $o$ - the **output gate** - decide what info is needed for current hidden state
    
<div>
    <table>
    <tr>
        <td><img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/hadamard_product.png?raw=1"></td><td style="text-align: left;"><b>Hadamard product:</b> bitwise multiplication</td>
    </tr>
    <tr>
        <td><img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/sigmoid.png?raw=1"></td><td style="text-align: left;"><b>Sigmoid activation:</b> pushes everything to 0 or 1</td>
    </tr>
    <tr>
        <td><img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/tanh.png?raw=1"></td><td style="text-align: left;"><b>Hyperbolic tangent activation:</b> pushes to 0 or 1, more like identity at the origin</td>
    </tr>
    </table>
</div>

Combining sigmoid with ⊙ has the effect of *masking* out information removing some, leaving others

## Gated Recurrent Unit

A **Gated Recurrent Unit** is a popular unit similar to LSTM, except more lightweight
* no output gate
* no context vector

Performance is often similar, but fewer parameters
* faster
* less memory


<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/RNN-vs-LSTM-vs-GRU.png?raw=1", width=700>
</div>

image source: http://dprogrammer.org/rnn-lstm-gru

## Let's work with some data

We'll do something that should be an easier learning problem: text classification with a recurrent network

<div>
    <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/RNN_classification.png?raw=1" width=700>
</div>

image source: SLP Fig. 9.8, https://web.stanford.edu/~jurafsky/slp3/9.pdf

### IMDB Reviews Dataset

In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split

dataset = load_dataset("imdb")
dataset

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
#uncomment these to work with a subset of the data
#data_subset_text, _, data_subset_label, _ = train_test_split(dataset["train"]["text"],dataset["train"]["label"],train_size=5000)
#train_data_text,  test_data_text, train_data_label, test_data_label = train_test_split(data_subset_text, data_subset_label,test_size = 0.2)

# uncomment these to use the full original data
train_data_text = dataset["train"]["text"]
train_data_label = dataset["train"]["label"]
test_data_text = dataset["test"]["text"]
test_data_label = dataset["test"]["label"]

In [ ]:
#printing out a sample review
print( train_data_text[125] )
print( train_data_label[125] )

I saw the capsule comment said "great acting." In my opinion, these are two great actors giving horrible performances, and with zero chemistry with one another, for a great director in his all-time worst effort. Robert De Niro has to be the most ingenious and insightful illiterate of all time. Jane Fonda's performance uncomfortably drifts all over the map as she clearly has no handle on this character, mostly because the character is so poorly written. Molasses-like would be too swift an adjective for this film's excruciating pacing. Although the film's intent is to be an uplifting story of curing illiteracy, watching it is a true "bummer." I give it 1 out of 10, truly one of the worst 20 movies for its budget level that I have ever seen.
0


### Importing libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SimpleRNN, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

### Preparing the data

In [ ]:
vocab_size = 10000
pad_length = 500

tokenizer = Tokenizer(num_words=vocab_size) #only keep the 10000 most common words
tokenizer.fit_on_texts(train_data_text)
tokenized_train_data = tokenizer.texts_to_sequences(train_data_text)
processed_train_data = pad_sequences(tokenized_train_data,maxlen=pad_length, padding='pre')
tokenized_test_data = tokenizer.texts_to_sequences(test_data_text)
processed_test_data = pad_sequences(tokenized_test_data,maxlen=pad_length, padding='pre')

train_target = np.array(train_data_label)
test_target = np.array(test_data_label)

**Important Note:** I originally had tried `padding='post'` which led to bad results
* having a bunch of 0s at the end of a sequence is really bad when you are only sending the last output to the next layer
* in general, we shouldn't be using post-padding with recurrent networks
 - unfortunately, this doesn't seem to be the problem with our encoder-decoder example, but it is still worth going back and fixing if you want to keep working with it

### Defining a simple LSTM-based architecture

Since this is a binary classification problem, we can use a sigmoid activation and `binary_crossentropy` loss.

In [ ]:
embedding_size = 50
hidden_layer_size = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=pad_length))
#model.add(Dropout(0.2))
model.add(LSTM(hidden_layer_size))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           500000    
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 560501 (2.14 MB)
Trainable params: 560501 (2.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model.fit(processed_train_data,
          train_target,
          epochs = 3,
          batch_size = 64,
          validation_data=(processed_test_data,test_target) )

Epoch 1/3
391/391 [==============================] - 82s 191ms/step - loss: 0.4275 - accuracy: 0.8024 - val_loss: 0.3184 - val_accuracy: 0.8678
Epoch 2/3
391/391 [==============================] - 42s 106ms/step - loss: 0.2614 - accuracy: 0.8964 - val_loss: 0.3578 - val_accuracy: 0.8540
Epoch 3/3
391/391 [==============================] - 31s 79ms/step - loss: 0.2141 - accuracy: 0.9181 - val_loss: 0.3437 - val_accuracy: 0.8728


## Exercise

The source I got this code from (https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/ ) included Dropout layers - you can try uncommenting them and see what it does.

It's also equivalent to writing

`model.add(LSTM(hidden_layer_size), dropout=0.2, recurrent_dropout=0.2)`

Do some searching and see what you can find out about what dropout layers do and why people use them. Discuss your findings with your group.

### Did dropout layers make a difference?
- Made Gonzalo's model worse
- Made my model very slightly better

### What do dropout layers do and why would you use them?

Drop out layers are designed to reduce overfitting by removing random nodes

In [ ]:
embedding_size = 50
hidden_layer_size = 100

dropout_model = Sequential()
dropout_model.add(Embedding(vocab_size, embedding_size, input_length=pad_length))
dropout_model.add(Dropout(0.2))
dropout_model.add(LSTM(hidden_layer_size))
dropout_model.add(Dropout(0.2))
dropout_model.add(Dense(1, activation='sigmoid'))

dropout_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(dropout_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 50)           500000    
                                                                 
 dropout (Dropout)           (None, 500, 50)           0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               60400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 560501 (2.14 MB)
Trainable params: 560501 (2.14 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
dropout_model.fit(processed_train_data,
          train_target,
          epochs = 3,
          batch_size = 64,
          validation_data=(processed_test_data,test_target) )

Epoch 1/3
391/391 [==============================] - 55s 130ms/step - loss: 0.4991 - accuracy: 0.7591 - val_loss: 0.3415 - val_accuracy: 0.8581
Epoch 2/3
391/391 [==============================] - 34s 86ms/step - loss: 0.2893 - accuracy: 0.8844 - val_loss: 0.3175 - val_accuracy: 0.8666
Epoch 3/3
391/391 [==============================] - 24s 61ms/step - loss: 0.2069 - accuracy: 0.9259 - val_loss: 0.3163 - val_accuracy: 0.8747


## Exercise

Run an experiment: What is the difference between using `SimpleRNN` and `LSTM` with this data?

In [ ]:
embedding_size = 50
hidden_layer_size = 100

rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size, embedding_size, input_length=pad_length))
rnn_model.add(Dropout(0.2))
rnn_model.add(SimpleRNN(hidden_layer_size))
rnn_model.add(Dropout(0.2))
rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(rnn_model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 500, 50)           500000    
                                                                 
 dropout_9 (Dropout)         (None, 500, 50)           0         
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 100)               15100     
                                                                 
 dropout_10 (Dropout)        (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 515201 (1.97 MB)
Trainable params: 515201 (1.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
rnn_model.fit(processed_train_data,
          train_target,
          epochs = 3,
          batch_size = 64,
          validation_data=(processed_test_data,test_target) )

Epoch 1/3
391/391 [==============================] - 247s 624ms/step - loss: 0.6977 - accuracy: 0.5252 - val_loss: 0.6552 - val_accuracy: 0.5858
Epoch 2/3
391/391 [==============================] - 233s 597ms/step - loss: 0.5717 - accuracy: 0.7047 - val_loss: 0.5233 - val_accuracy: 0.7409
Epoch 3/3
391/391 [==============================] - 228s 583ms/step - loss: 0.4361 - accuracy: 0.8019 - val_loss: 0.4392 - val_accuracy: 0.8122


## Applied Exploration

Do one of the following:

1. Redo your experiment with another classification dataset. Choose something with more than 2 classes - this will be good practice is understanding the difference you need to make to the model and data prep. Describe your data and results as usual.
    * I also suggest including a GRU layer in your experiment as well: https://keras.io/api/layers/recurrent_layers/gru/

2. Edit the Encoder-Decoder code from last time to use LSTM or GRU.
    * Note that since LSTM returns both a context and hidden state, you will get an output, a hidden state, and context returned from the LSTM layer (instead of just the output and state). It will look something like
    

In [ ]:
encoder_rnn = LSTM(100, return_state=True)
encoder_outputs, state_h, state_c = encoder_rnn(enc_emb)

NameError: ignored

and you will pass both state_h, state_c as the *context vector* which is the initial state for the decoder. See the source from last time to flesh out the example: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [ ]:
def solution(plaintext):
    encrypted_text = ""
    for char in plaintext:
        if char == " ":
            encrypted_text += "0"*6
        else:
            if char != char.lower():
                encrypted_text += "000001"

            char = char.lower()

            if char == "w":
                encrypted_text += "010111"
            else:
                if char in ["x","y","z"]:
                    char = char(ord(chr)-1)
                if ord(char)%10 == ord("a")%10:
                    encrypted_text += "10"

                    dif = int((ord(char)-ord("a")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "00"

                    encrypted_text += rank(dif)

                elif ord(char)%10 == ord("b")%10:
                    encrypted_text += "11"

                    dif = int((ord(char)-ord("b")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "00"

                    encrypted_text += rank(dif)

                elif ord(char)%10 == ord("c")%10:
                    encrypted_text += "10"

                    dif = int((ord(char)-ord("c")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "10"

                    encrypted_text += rank(dif)

                elif ord(char)%10 == ord("d")%10:
                    encrypted_text += "10"

                    dif = int((ord(char)-ord("d")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "11"

                    encrypted_text += rank(dif)
                elif ord(char)%10 == ord("e")%10:
                    encrypted_text += "10"

                    dif = int((ord(char)-ord("e")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "01"

                    encrypted_text += rank(dif)

                elif ord(char)%10 == ord("f")%10:
                    encrypted_text += "11"

                    dif = int((ord(char)-ord("f")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "10"

                    encrypted_text += rank(dif)
                elif ord(char)%10 == ord("g")%10:
                    encrypted_text += "11"

                    dif = int((ord(char)-ord("g")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "11"

                    encrypted_text += rank(dif)
                elif ord(char)%10 == ord("h")%10:
                    encrypted_text += "11"

                    dif = int((ord(char)-ord("h")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "01"

                    encrypted_text += rank(dif)
                elif ord(char)%10 == ord("i")%10:
                    encrypted_text += "01"

                    dif = int((ord(char)-ord("i")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "10"

                    encrypted_text += rank(dif)
                elif ord(char)%10 == ord("j")%10:
                    encrypted_text += "01"

                    dif = int((ord(char)-ord("j")) / 10)
                    encrypted_text += rank(dif)
                    dif = dif - 1

                    encrypted_text += "11"

                    encrypted_text += rank(dif)
    return encrypted_text

def rank(dif):
    if dif > 0:
        return "1"
    else:
        return "0"